In [ ]:
# Misc imports
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')
import torch
torch.set_printoptions(linewidth=200)
import seaborn as sns
sns.set_style("darkgrid")
import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
       '/storage'
))
import yaml
from pathlib import Path
# Local imports
from ese.experiment.analysis.run_inference import get_cal_stats
from ese.experiment.analysis.analysis_utils.submit_utils import get_ese_inference_configs
# Ionpy imports
from ionpy.util import Config
from ionpy.analysis import ResultsLoader

# Define some useful paths.
scratch_root = Path("/storage/vbutoi/scratch/ESE")
code_root = Path("/storage/vbutoi/projects/ESE")
rs = ResultsLoader()

# For using code without restarting.
%load_ext autoreload
%autoreload 2
# For using yaml configs.
%load_ext yamlmagic

In [ ]:
%%yaml inference_config

experiment:
    exp_root: '?'
    seed: '?'
    supports_per_target: '?' 
    threshold: 0.5 
    fixed_support_sets: True 

log:
    root: '?'
    min_fg_pixels: 0 
    log_interval: 20 
    log_image_stats: True 
    log_pixel_stats: True 
    gether_inference_stats: True
    track_ensemble_member_scores: False 
    summary_compute_global_metrics: True

dataloader:
    batch_size: 1 
    num_workers: 0
    pin_memory: True 

In [ ]:
%%yaml calibration_cfg 

calibrator:
    _name: '?'
    normalize: False

local_calibration:
    num_prob_bins: 15
    neighborhood_width: 3
    square_diff: False 

global_calibration:
    num_prob_bins: 15
    num_classes: 2
    neighborhood_width: 3
    square_diff: False 
    loss_weights: None 

In [ ]:
%%yaml incontext_model_cfg 

# # For incontext datasets
####################################
data:
    splits: ('val',)
    support_size: '?' 
    preload: False 

ensemble:
    combine_fn: '?'
    combine_quantity: '?'
    normalize: False

model:
    _type: "incontext"
    pretrained_exp_root : None
    checkpoint: "max-val_od-dice_score"

In [ ]:
%%yaml incontext_submit_cfg_args

submit_opts:
    ens_cfg_options:
        - ('mean', 'probs')
    num_ens_membs:
        - 1
        # - 2
        # - 4
        # - 8
        # - 16
        # - 32

In [ ]:
exp_name = f'04_29_24_SpineWeb_ExampleImportance'
group_dict = {
    "datasets": [
        # "ACDC", 
        # "PanDental_Jaw", 
        # "PanDental_Mandible", 
        "SpineWeb", 
        # "STARE", 
        # "WBC"
    ],
    "exp_group": exp_name,
    "model_type": "incontext",
    "base_models_group": "/storage/jjgo/results/omni/2023-02-16_Universeg-HO_Adrian/20230214_112221-1C7K-2cb972b55f72d56b3842c2d2d8bdd3c6",
    "scratch_root": "/storage/vbutoi/scratch/ESE",
    "inf_cfg_root": "/storage/vbutoi/projects/ESE/ese/experiment/configs/inference",
}

## Gather Inference Options.

In [ ]:
%%yaml inference_config_opts

calibrator:
    - Uncalibrated

data.support_size:
    - 8

experiment.seed:
    - 40
    - 41
    - 42
    - 43

experiment.supports_per_target:
    - 16 

In [ ]:
# Load the inference cfg from local.
##################################################
inf_cfg_root = code_root / "ese" / "experiment" / "configs" / "inference"

##################################################
with open(inf_cfg_root / "Calibration_Metrics.yaml", 'r') as file:
    cal_metrics_cfg = yaml.safe_load(file)

##################################################
base_cfg = Config(inference_config).update([calibration_cfg, cal_metrics_cfg, incontext_model_cfg])

In [ ]:
inference_func = get_cal_stats
# Get the configs for the different runs.
inf_cfgs = get_ese_inference_configs(
    group_dict=group_dict,
    base_cfg=base_cfg,
    inf_cfg_opts=inference_config_opts,
    base_cfg_args=incontext_submit_cfg_args
)

In [ ]:
len(inf_cfgs)

## Running Jobs

In [ ]:
from ese.experiment.analysis.run_inference import get_cal_stats

In [ ]:
from ese.experiment.experiment import run_ese_exp

###### Run individual jobs
run_ese_exp(
    config=inf_cfgs[0], 
    job_func=get_cal_stats,
    run_name='debug',
    show_examples=True,
    gpu='3',
) 

In [ ]:
# from ese.experiment.experiment import submit_ese_exps 

# #### Run Batch Jobs
# submit_ese_exps(
#     config_list=inf_cfgs,
#     job_func=get_cal_stats,
#     available_gpus=[
#         '0', 
#         '1', 
#         '2', 
#         '3'
#     ]
# )